In [390]:
from bs4 import BeautifulSoup as bsp
import requests
import os
import json
import lxml
import time
import random
import re
import numpy as np
import pandas as pd
import csv

In [391]:
quizplease_main_url = 'https://quizplease.ru'
classic_game_teg = 'Квиз, плиз!'
headers = {
        'accept': '*/*',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
    }

### 1. Выгрузка страницы с раздела расписание прошедших игр

In [8]:
games_links_dict = {}
classic_games_dict = {} #словарь для разбивки ссылок по страницам, которых данные ссылки были найдены
classic_games_on_page = []
last_page_number = 262

#парсинг ссылок на классические игры со страницы расписания прошедших игр
for page_number in range(1, last_page_number + 1):
    #url_schedule_page = f'https://quizplease.ru/schedule-past?page={page_number}&per-page=12'
    #page_on_schedule = requests.get(url_schedule_page, headers=headers)
    #src_schedule = page_on_schedule.text
    game_number = 0

    #with open(f'data/schedule-past_{page_number}.html', 'w', encoding='utf-8') as output_file:
        #output_file.write(src_schedule)

    #работа со скаченной страницей
    with open(f'data/schedule-past_{page_number}.html', encoding='utf-8') as input_file:
        working_src = input_file.read()
    soup = bsp(working_src, 'lxml')

    #получение списка ссылок на странице прошедших игр
    games_links = soup.find_all(class_='schedule-block-head w-inline-block')
    
    for link in games_links:
        game_link_name = link.find(class_='h2-game-card').text.strip()
        game_short_href = link.get('href')
        game_full_href = quizplease_main_url + game_short_href
        games_links_dict[game_full_href] = game_link_name
        
    for key in games_links_dict.keys():
        if games_links_dict[key] == classic_game_teg:
            classic_games_on_page.append(key)
    
    classic_games_dict[page_number] = list(classic_games_on_page)
    if page_number % 20 == 0 and page_number != last_page_number:
        print(f'Страница {page_number} готова, осталось {last_page_number - page_number}')
    elif page_number == last_page_number - 1:
        print(f'Страница {page_number} готова, осталась последняя страница')
    #time.sleep(random.randrange(1, 2))
    classic_games_on_page.clear()
    games_links_dict.clear()
print('Готово')

Страница 20 готова, осталось 242
Страница 40 готова, осталось 222
Страница 60 готова, осталось 202
Страница 80 готова, осталось 182
Страница 100 готова, осталось 162
Страница 120 готова, осталось 142
Страница 140 готова, осталось 122
Страница 160 готова, осталось 102
Страница 180 готова, осталось 82
Страница 200 готова, осталось 62
Страница 220 готова, осталось 42
Страница 240 готова, осталось 22
Страница 260 готова, осталось 2
Страница 261 готова, осталась последняя страница
Готово


### 2. Составление словаря с краткой информацией об играх

In [516]:
count = 0
schedule_pages_list = [val for val in os.listdir('data') if os.path.isfile(f'data/{val}')] #список файлов-страниц в папке data
games_dict = [] #основной словарь с играми
games_blocks_list = [] #список всех блоков с играми на странице
GAME_TYPES = ['Классические игры', 'Кино и музыка', 'Стрим', 'Teens']
for schedule_page in schedule_pages_list:
    
    with open(f'data/{schedule_page}', encoding='utf-8') as input_file:
        src = input_file.read()
    soap = bsp(src, 'lxml')
    
    #данные для объекта списка
    game_link = 'Нет'
    game_name = 'Нет'
    game_type = 'Нет'#на странице игры class_='tag_name'
    package_number = 0
    place = 'Нет'
    beginning_time = 'Нет'
    game_date = 'Нет'
    day_of_the_week = 'Нет'
    cost = 0
    total_table = []
    game_link_id = 'Нет'
    
    games_blocks_list = soap.find_all(class_='schedule-block-top')
    for game_block in games_blocks_list:
        count += 1
        game_link = quizplease_main_url + game_block.find('a').get('href')
        game_name = game_block.find('div', class_='h2 h2-game-card h2-left').text.strip()
        package_number = game_block.find('div', class_='h2 h2-game-card h2-left').find_next_sibling().text.strip()
        package_number = package_number.replace('#', '') #замена лишних знаков
        place = game_block.find('div', class_='schedule-block-info-bar').text.strip()
        place = place.replace(' Где это?', '')
        beginning_time = game_block.find('div', class_='schedule-info').find_next_sibling().find(class_='techtext').text.strip()
        beginning_time = beginning_time.replace('в ', '') #замена лишних знаков
        game_date_and_day_of_the_week = game_block.find_parent(class_='schedule-column').find(class_='h3 h3-white h3-mb10').text.strip().split(', ')
        day_of_the_week = game_date_and_day_of_the_week[1]
        game_date = game_date_and_day_of_the_week[0]
        cost = game_block.find('div', class_='schedule-info last').find('div', class_='text').text.strip().split(', ')[0]
        game_link_id = f'{re.search(ptrn_link_id, game_link).group(0)}_{package_number}'
        
        games_dict.append(
            {
                'ID_Game': game_link_id,
                'Ссылка на игру': game_link,
                'Тип игры': game_type,
                'Название игры': game_name,
                'Номер пакета': package_number,
                'Место': place,
                'Время начала': beginning_time,
                'Дата': game_date,
                'День недели': day_of_the_week,
                'Стоимость': cost,
                'Итоговая таблица': total_table
            }
        )

### 3. Выгрузка страниц по ссылкам game_link

In [547]:
ptrn_link_id = '\d+$'
count = 0
with open('data/games_short_info.json', encoding='utf-8') as input_file:
    loading_games_info_json = json.loads(input_file.read())

for list_value in loading_games_info_json[101:201]:
    count += 1
    link = list_value['Ссылка на игру']
    package_number = list_value['Номер пакета']
    src = requests.get(link, headers=headers).text
    game_link_id = re.search(ptrn_link_id, link).group(0)
    
    
    with open(f'data/game_pages/{game_link_id}_{package_number}.html', 'w', encoding='utf-8') as output_file:
        output_file.write(src)
    time.sleep(random.randrange(1, 3))
    print(f'{count} - Создана страница {list_value["Название игры"]} от {list_value["Дата"]}')
print('Готово')

1 - Создана страница Квиз, плиз! от 26 ноября
2 - Создана страница Кино и музыка [стрим] от 26 ноября
3 - Создана страница [верните мой 2007] от 25 ноября
4 - Создана страница Квиз, плиз! от 25 ноября
5 - Создана страница Квиз, плиз! от 25 ноября
6 - Создана страница ОТКРЫТИЕ ЗИМНЕГО СЕЗОНА [полуфинал] от 25 ноября
7 - Создана страница ОТКРЫТИЕ ЗИМНЕГО СЕЗОНА [полуфинал] от 25 ноября
8 - Создана страница [кино и музыка] от 24 ноября
9 - Создана страница Квиз, плиз! от 24 ноября
10 - Создана страница [новички] от 23 ноября
11 - Создана страница Квиз, плиз! от 23 ноября
12 - Создана страница [верните мой 2007] от 22 ноября
13 - Создана страница Квиз, плиз! от 21 ноября
14 - Создана страница Квиз, плиз! от 21 ноября
15 - Создана страница [кино и музыка] от 21 ноября
16 - Создана страница Квиз, плиз! от 20 ноября
17 - Создана страница Квиз, плиз! [стрим] от 20 ноября
18 - Создана страница Квиз, плиз! от 20 ноября
19 - Создана страница музыка [шазам] рашн эдишн от 20 ноября
20 - Создана стр

### 4. Добавление итоговой таблицы + типа игры в словарь

In [548]:
games_pages_list = [val for val in os.listdir('data/game_pages')] #список файлов-страниц игр в папке game_pages

for game_page in games_pages_list:
    
    with open(f'data/game_pages/{game_page}', encoding='utf-8') as input_file:
        src_game_page = input_file.read()
    
    game_page_id = game_page.replace('.html', '')
    soap = bsp(src_game_page, 'lxml')
    game_type = soap.find(class_='game-tag').text.strip()
    total_table = {}
    try:
        #Парсинг заголовков
        game_table_headers = [val.text.strip() for val in soap.find('div', class_='game-table-block').find('thead').find_all('td', class_='label game-table__header-cell')]
        #Парсинг тела таблицы
        game_table_body = soap.find('div', class_='game-table-block').find('thead').find_next_siblings()
    except Exception as err:
        print(f'{err} : {game_page_id}')
    #Заполнение total_table пустыми списками
    for header in game_table_headers:
        total_table[header] = []
    
    
    
    #заполнение пустых списоков из total_table значениями из таблицы
    for tr in game_table_body:
        td_list = tr.find_all('td')
        
        for i in range(len(td_list)):
            td_text = td_list[i].text.strip()
            try:
                total_table[game_table_headers[i]].append(td_text)
            except Exception as err:
                print(f'{err} : {game_page_id} : {td_list}.\nЧто-то пошло не так')
                print(*td_list[i], sep="\n")
    
    #добавление Итоговой таблицы и Типа игры games_dict
    count = 0
    for obj in games_dict:
        if obj['ID_Game'] == game_page_id:
            count += 1
            obj['Тип игры'] = game_type
            obj['Итоговая таблица'] = total_table
            #print(f'Объект {game_page} обновлен {count} раз, проверка правильности объекта: {game_page_id == obj["ID_Game"]}')

print('Готово')

list index out of range : 32361_592 : [<td class="game-table__cell text">15</td>, <td class="game-table__cell text"></td>, <td class="game-table__cell text">Сырки Б.Ю. Александров</td>, <td class="game-table__cell text">5</td>, <td class="game-table__cell text">3</td>, <td class="game-table__cell text">6</td>, <td class="game-table__cell text">6</td>, <td class="game-table__cell text">6</td>, <td class="game-table__cell text">2</td>, <td class="game-table__cell text">8</td>, <td class="game-table__cell text">36</td>, <td class="game-table__cell text"> </td>].
Что-то пошло не так
 
list index out of range : 32365_593 : [<td class="game-table__cell text">15</td>, <td class="game-table__cell text"></td>, <td class="game-table__cell text">Пила-45</td>, <td class="game-table__cell text">3</td>, <td class="game-table__cell text">3</td>, <td class="game-table__cell text">2</td>, <td class="game-table__cell text">6</td>, <td class="game-table__cell text">3</td>, <td class="game-table__cell tex

Готово


### Загрузка Итоговых таблиц в excel

In [549]:
for obj in games_dict:
    if obj['Итоговая таблица'] != []:
        df_obj = pd.DataFrame(obj['Итоговая таблица'])
        df_obj.to_excel(f'data/total_tables/{obj["Название игры"]}_{obj["ID_Game"]}.xlsx', index=False)

### Создание json файла

In [550]:
#создание json файла
with open('data/games_short_info.json', 'w', encoding='utf-8') as output_file:
    json.dump(games_dict, output_file, indent=4, ensure_ascii=False)

## Работа с файлами games_dict/games_short_info.json

In [574]:
total_table_list = [pd.read_excel(f'data/total_tables/{file}') for file in os.listdir('data/total_tables') if 'Квиз, плиз!' in file and '[Стрим]' not in file]
main_df_quiz = pd.DataFrame()
local_table_df = ''

total_table_list[1]
#print(*total_table_list, sep='\n')

,Место,Город,Название команды,1 раунд,2 раунд,3 раунд,4 раунд,5 раунд,6 раунд,7 раунд,итого
0,1,Москва,Пластилиновая корона,6,6,6,12,6,6,18,60
1,2,Саратов,Кто бы знал,6,5,6,12,6,4,18,57
2,3,Саратов,BeerManGame,6,6,6,12,6,4,13,53
3,4,Москва,Скрежет Зу,5,5,6,12,6,4,13,51
4,5,Санкт-Петербург,P.S. vrt,4,5,5,12,6,2,16,50
5,6,Москва,Тимур и его команда,6,6,4,11,6,4,13,50
6,7,Чебоксары,8 пятниц,5,5,5,8,6,4,16,49
7,8,Москва,Без пятнадцати девятнадцать,5,3,6,12,6,2,15,49
8,9,Москва,Шекспира хвост,5,5,6,11,6,2,14,49
9,10,Казань,А где все?,6,4,6,11,6,4,12,49
